In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webdriver-manager

  Using cached urllib3-1.23-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.7-py2.py3-none-any.whl (58 kB)
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.14
    Uninstalling urllib3-1.26.14:
      Successfully uninstalled urllib3-1.26.14
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.8.2 requires urllib3[socks]~=1.26, but you have urllib3 1.23 which is incompatible.


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from selenium.webdriver.common.keys import Keys
import praw
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [5]:
def scrape_url(source_path):
    page = 1
    url = []
    search_list = ['singapore','bivalent']
    while(page <= 2): 
        browser = webdriver.Chrome('chromedriver',options=chrome_options)
        browser.get(source_path)
        Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        source = browser.page_source
        soup = bs(source, 'html.parser')
        body = soup.find('body')

        threads = soup.find_all("header", class_="search-result-header")
        next_page = soup.find_all("span", class_="nextprev")
        source_path = next_page[0].find("a").get('href')
                    
        for link in threads:
            title = link.find("a").text
            if re.compile('(?=.*' + ')(?=.*'.join(search_list) + ')', re.IGNORECASE).search(title): #re.IGNORECASE is used to ignore case
                url_string = link.find('a').get('href')
                url.append(url_string)
        page += 1            
    return url

In [6]:
source_path = "https://old.reddit.com/search?q=singapore+bivalent"
url_list = scrape_url(source_path)

In [7]:
url_list

['https://old.reddit.com/r/CoronavirusDownunder/comments/10bwiuy/singapore_data_for_bivalent_covid19_vaccines_does/',
 'https://old.reddit.com/r/Coronavirus/comments/10bty8k/singapore_data_for_bivalent_covid19_vaccines_does/',
 'https://old.reddit.com/r/singapore/comments/y1162c/bivalent_moderna_covid19_vaccine_available_in/',
 'https://old.reddit.com/r/ChannelNewsAsia/comments/10bp7v2/singapore_data_for_bivalent_covid19_vaccines_does/',
 'https://old.reddit.com/r/SingaporeRaw/comments/y8qdwm/bivalent_vaccines_for_1849_age_group_to_come/',
 'https://old.reddit.com/r/ChannelNewsAsia/comments/zjpk1t/386000_bivalent_covid19_vaccines_given_out_in/',
 'https://old.reddit.com/r/singapore/comments/yd1hv4/singapore_grants_interim_authorisation_for_pfizer/',
 'https://old.reddit.com/r/singapore/comments/xetlo0/new_bivalent_covid19_booster_vaccine_doses/',
 'https://old.reddit.com/r/corona_pandemic/comments/yn66ev/singapore_which_has_one_of_the_worlds_highest/',
 'https://old.reddit.com/r/Channe

In [8]:
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='', \
                     username='', \
                     password='')

In [9]:
topics_dict = { "author": [],
                "submission" : [],
                "body": [],
                "score": [],
                "id": [],
                "parent_id": [],
                "created": []}

In [10]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [11]:
def process_topic_data(submission):
    comments = []
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        timestamp = get_date(comment.created)
        topics_dict["author"].append(comment.author)
        topics_dict["submission"].append(comment.submission)
        topics_dict["body"].append(comment.body)
        topics_dict["score"].append(comment.score)
        topics_dict["id"].append(comment.id)
        topics_dict["parent_id"].append(comment.parent_id)
        topics_dict["created"].append(timestamp)
    return comments

In [12]:
for link in url_list:
    submission = reddit.submission(url=link)
    process_topic_data(submission)
#     collection_list.append(data)

In [13]:
topics_data = pd.DataFrame(topics_dict)

In [14]:
topics_data

,author,submission,body,score,id,parent_id,created
0,spaniel_rage,10bwiuy,The CDC seems to think it's unlikely this safe...,11,j4cmwjf,t3_10bwiuy,2023-01-15 03:42:02
1,None,10bwiuy,[removed],2,j4g1gsv,t3_10bwiuy,2023-01-15 21:43:34
2,None,10bwiuy,[removed],1,j4dhd5o,t3_10bwiuy,2023-01-15 07:07:23
3,pharmaboy2,10bwiuy,I’m a little troubled that the CDC reported th...,6,j4f92fq,t1_j4cmwjf,2023-01-15 15:51:30
4,mrspc1000,10bwiuy,I would not trust a large pharmaceutical compa...,11,j4eih22,t1_j4cmwjf,2023-01-15 11:33:05
5,sacre_bae,10bwiuy,Good,4,j4crho1,t1_j4cmwjf,2023-01-15 04:12:58
6,Mymerrybean,10bwiuy,[Pfizer Document 5.3.6](https://www.phmpt.org/...,0,j4g071f,t1_j4cmwjf,2023-01-15 21:31:43
7,None,10bwiuy,[removed],0,j4erd83,t1_j4cmwjf,2023-01-15 12:42:35
8,CoronavirusDownunder-ModTeam,10bwiuy,Thank you for contributing to r/CoronavirusDow...,1,j4i3i9d,t1_j4g1gsv,2023-01-16 05:52:26
9,CoronavirusDownunder-ModTeam,10bwiuy,Thank you for contributing to r/CoronavirusDow...,2,j4dirwv,t1_j4dhd5o,2023-01-15 07:16:56


In [25]:
topics_data = topics_data.dropna()

In [26]:
topics_data.to_csv('Reddit.csv', index=False)